# Julia Installation

Based on [this repository](https://github.com/Dsantra92/Julia-on-Colab).

In [1]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

In [2]:
println("Julian working")

Julian working


# Import packages

In [1]:
import Pkg; Pkg.add("MAT")
using LinearAlgebra, MAT, DelimitedFiles

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed OpenSSL_jll ──────── v1.1.19+0
   Installed TranscodingStreams ─ v0.9.10
   Installed Requires ─────────── v1.3.0
   Installed HDF5_jll ─────────── v1.12.2+2
   Installed BufferedStreams ──── v1.1.0
   Installed Compat ───────────── v4.5.0
   Installed CodecZlib ────────── v0.7.0
   Installed HDF5 ─────────────── v0.16.13
   Installed MAT ──────────────── v0.10.3
    Updating `~/.julia/environments/v1.7/Project.toml`
  [23992714] + MAT v0.10.3
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [e1450e63] + BufferedStreams v1.1.0
  [944b1d66] + CodecZlib v0.7.0
  [34da2185] + Compat v4.5.0
  [f67ccb44] + HDF5 v0.16.13
  [23992714] + MAT v0.10.3
  [ae029012] + Requires v1.3.0
  [3bb67fe8] + TranscodingStreams v0.9.10
  [0234f1f7] + HDF5_jll v1.12.2+2
  [458c3c95] + OpenSSL_jll v1.1.19+0
  [37e2e46d] + LinearAlgebra
  [2f01184e] + SparseArrays
  [e66e0078] + CompilerSupportLibr

# Auxiliary Functions

In [81]:
function get_time_in_ms()
    return convert(Dates.Millisecond, Dates.now())
end

get_time_in_ms (generic function with 1 method)

In [82]:
start_time = 0;

In [83]:
function ldet(A)
    (value,sign_det) = logabsdet(A);
    if (sign_det > 0)
        return value
    else
        return -Inf
    end
end

function ldet_objval(A,x)
    return ldet(A'*diagm(vec(x))*A);
end

ldet_objval (generic function with 1 method)

In [84]:
function getMatlabInstance(instanceName,matrixName)
    myInst = string(instanceName,".mat");
    file = matopen(string(myInst));
    A = read(file, string(matrixName)); # note that this does NOT introduce a variable ``varname`` into scope
    close(file);
    return A
end

function saveMatlabInstance(instanceName,matrixName,matrix)
    if !isdir("ResultsInstances")
        mkdir("ResultsInstances")
    end
    myInst = string("ResultsInstances/",instanceName,".mat");
    matwrite(myInst,Dict(matrixName => matrix));
    return
end

function writeTXT(out_name,notes)
    if !isdir("ResultsTXT")
        mkdir("ResultsTXT")
    end
    outfile = string("ResultsTXT/",out_name);
    f = open(outfile, "w");
    for i in eachindex(notes)
        println(f, notes[i]);
    end
    close(f);
end

writeTXT (generic function with 1 method)

# Local Search Functions

## Initial Solutions

In [85]:
function initBinary(A,R,s,m,n)
    F = svd(A,full= true);
    U = F.U;
    x = zeros(n,1);
    for j = (1:n)
       for i = (1:s) 
           x[j] = x[j] + U[j,i]^2; 
       end
    end
    x_save = copy(x);
    x = zeros(n,1);
    for row in R
        x[row] = 1;
        x_save[row] = 0;
    end
    for i = (1:s-m)
        max_indice = argmax(x_save);
        x[max_indice] = 1;
        x_save[max_indice] = 0;
    end
    zlb = ldet_objval(A,x);
    xlb = x;  
    return xlb,zlb
end

function initGreedy(A,R,s,m,n)
    F = svd(A,full= true);
    U = F.U;
    S = F.S;
    x = zeros(n,1);
    k = min(s,m);
    for j = (1:n)
       for i = (1:k)
           x[j] = x[j] + (S[i]*U[j,i])^2; 
       end
    end
    x_save = copy(x);
    x = zeros(n,1);
    for row in R
        x[row] = 1;
        x_save[row] = 0;
    end
    for i = (1:s-m)
        max_indice = argmax(x_save);
        x[max_indice] = 1;
        x_save[max_indice] = 0;
    end
    zlb = ldet_objval(A,x);
    xlb = x;  
    return xlb,zlb
end

initGreedy (generic function with 1 method)

## Local Searches

In [86]:
function LSFI(A,n,x_init,z_lb) # Local Search First Improvement
    x = copy(x_init);
    flag = true;
    while flag
        flag = false;
        for i = (1:n)
            if x[i] > 0
                x[i] = 0;
                for j = (1:n) 
                    if j != i && x[j] == 0
                        x[j] = 1;
                        z_lb_new = ldet_objval(A,x);
                        if z_lb_new > z_lb
                            z_lb = z_lb_new;
                            flag = true;
                            break 
                        else
                            x[j] = 0;
                        end
                    end
                end
                if flag
                    break 
                else
                    x[i] = 1;
                end
            end
        end
    end
    return x,z_lb
end


function LSFP(A,n,x_init,z_lb) # Local Search First Improvement Plus
    x = copy(x_init);
    flag = true;
    leave_x,enter_x = 0,0;
    while flag
        flag = false;
        for i = (1:n)
            if x[i] > 0
                x[i] = 0;
                for j = (1:n) 
                    if j != i && x[j] == 0
                        x[j] = 1;
                        z_lb_new = ldet_objval(A,x);
                        if z_lb_new > z_lb
                            leave_x = i;
                            enter_x = j;
                            z_lb = z_lb_new;
                            flag = true; 
                        end
                        x[j] = 0;
                    end
                end
                if flag
                    break 
                else
                    x[i] = 1;
                end
            end
        end
        if flag
            #x[leave_x] = 0;
            x[enter_x] = 1;
        end
    end
    return x,z_lb
end

function LSBI(A,n,x_init,z_lb) # Local Search First Improvement Plus
    x = copy(x_init);
    flag = true;
    leave_x,enter_x = 0,0;
    while flag
        flag = false;
        for i = (1:n)
            if x[i] > 0
                x[i] = 0;
                for j = (1:n) 
                    if j != i && x[j] == 0
                        x[j] = 1;
                        z_lb_new = ldet_objval(A,x);
                        if z_lb_new > z_lb
                            leave_x = i;
                            enter_x = j;
                            z_lb = z_lb_new;
                            flag = true; 
                        end
                        x[j] = 0;
                    end
                end
                x[i] = 1;
            end
        end
        if flag
            x[leave_x] = 0;
            x[enter_x] = 1;
        end
    end
    return x,z_lb
end

LSBI (generic function with 1 method)

In [87]:
function runLocalSearch(A,R,n,m,s)
    # Initial solution
    t_init_bin = @elapsed x_init_bin,z_init_bin = initBinary(A,R,s,m,n);
    t_init_gdy = @elapsed x_init_gdy,z_init_gdy = initGreedy(A,R,s,m,n);
    # Get initial results
    X = [x_init_bin,x_init_gdy];
    Z = [z_init_bin,z_init_gdy];
    T = [t_init_bin,t_init_gdy];
    # Heuristics
    X_init = [x_init_bin,x_init_gdy];
    for x_init in X_init
        z_init = ldet_objval(A,x_init);
        t_FI   = @elapsed x_FI, z_FI = LSFI(A,n,x_init,z_init);
        println("LSFI: $(z_FI) Time: $(t_FI)");# Solution: $(x_FI)");
        t_FP   = @elapsed x_FP, z_FP = LSFP(A,n,x_init,z_init);
        println("LSFP: $(z_FP) Time: $(t_FP)");# Solution: $(x_FP)");
        t_BI   = @elapsed x_BI, z_BI = LSBI(A,n,x_init,z_init);
        println("LSBI: $(z_BI) Time: $(t_BI)");# Solution: $(x_BI)");
        push!(X,x_FI,x_FP,x_BI);
        push!(Z,z_FI,z_FP,z_BI);
        push!(T,t_FI,t_FP,t_BI);
    end
    z_heur = maximum(Z);
    indsX  = findall(Z .== z_heur); # we can receive more than one indice here
    x_heur = X[indsX[1]];
    # More informations
    sum_x = [];
    max_x = [];
    for el in X
        push!(sum_x,sum(el));
        push!(max_x,maximum(el));
    end
    # All Informations
    infos = (Z,T,sum_x,max_x,indsX);
    return x_heur, z_heur,infos
end

runLocalSearch (generic function with 1 method)

# Run code

In [88]:
function getMatlabInitialInfos(n,i)
    nameInst = string("Instance_",n,"_",i);
    A = getMatlabInstance(nameInst,"A");
    R = trunc.(Int,getMatlabInstance(nameInst,"R"));
    s = trunc(Int,n/2);
    m = trunc(Int,n/4);
    return A,R,m,s
end

getMatlabInitialInfos (generic function with 1 method)

In [89]:
function writeAllInfos(Results,InfoLS,info_ls,results)
    if !isdir("ResultsCSV")
        mkdir("ResultsCSV")
    end
    # File name
    resultsstr = string("ResultsCSV/results.csv");
    zLSstr     = string("ResultsCSV/zLS.csv");
    tLSstr     = string("ResultsCSV/tLS.csv");
    sum_str    = string("ResultsCSV/sumLS.csv");
    max_str    = string("ResultsCSV/maxLS.csv");
    # Get infos
    z       = info_ls[1];
    t       = info_ls[2];
    sum_x   = info_ls[3];
    max_x   = info_ls[4];
    # push new informations
    push!(Results,results);
    push!(InfoLS[1],z);
    push!(InfoLS[2],t);
    push!(InfoLS[3],sum_x);
    push!(InfoLS[4],max_x);
    # save results
    writedlm(resultsstr,  Results, ',')
    writedlm(zLSstr,InfoLS[1], ',');
    writedlm(tLSstr,InfoLS[2], ',');
    writedlm(sum_str,InfoLS[3], ',');
    writedlm(max_str,InfoLS[4], ',');
    return Results,InfoLS
end

writeAllInfos (generic function with 1 method)

In [90]:
function saveFiles(n,i,x_ls)
    x_ls_name   = string("x_ls_",n,"_",i);
    saveMatlabInstance(x_ls_name,"x_ls",x_ls);
    return
end

saveFiles (generic function with 2 methods)

In [91]:
pwd()

"/content"

In [92]:
readdir()

8-element Vector{String}:
 ".\\ResultsInstances"
 ".\\ResultsInstances\\x_ls_250.mat"
 ".config"
 ".ipynb_checkpoints"
 "Instance_250_dense.mat"
 "Instance_250_sparse.mat"
 "_ResultsCSV"
 "sample_data"

In [93]:
using Dates

In [94]:
now()

2022-12-24T18:14:46.741

In [97]:
Results = [];
InfoLS  = [[],[],[],[]];
InfoLS_old  = [[],[],[],[]];
my_N = [250];
#instances = ["Instance_250_s", "Instance_250_d"]
instType = ["dense", "sparse"]

println(now())
for n in my_N
    for i in instType
        #A,R,m,s = getMatlabInitialInfos(n,i);
        nameInst = string("Instance_", n, "_", i);
        A = getMatlabInstance(nameInst,"A");
        R = trunc.(Int,getMatlabInstance(nameInst,"R"));
        s = 150;
        m = 50;
        
        #start_time = get_time_in_ms()
        time_ls = @elapsed x_ls, z_ls,info_ls = runLocalSearch(A,R,n,m,s);
        println("$(n), $(i): Finished LS $(z_ls) with $(time_ls) sec")
        flush(stdout); # show println
        results  = [n,m,s,i,z_ls,time_ls];
        Results,InfoLS = writeAllInfos(Results,InfoLS,info_ls,results);
        saveFiles(n,i,x_ls)
    end
end
println(now())

2022-12-24T18:15:05.382
LSFI: 249.72454326276616 Time: 296.311053948
LSFP: 249.74645879263633 Time: 372.582533748
LSBI: 249.74645879263633 Time: 836.123767002
LSFI: 249.73568129052666 Time: 501.12551052
LSFP: 249.73568129052666 Time: 445.386692025
LSBI: 249.7330140509374 Time: 830.261731261
250, dense: Finished LS 249.74645879263633 with 3282.276662195 sec
LSFI: 19.106746218102483 Time: 217.073843994
LSFP: 19.106746218102483 Time: 175.504449322
LSBI: 19.106746218102483 Time: 671.982092724
LSFI: 19.106746218102483 Time: 309.477909886
LSFP: 19.106746218102483 Time: 249.052866989
LSBI: 19.106746218102483 Time: 420.072959743
250, sparse: Finished LS 19.106746218102483 with 2043.176871386 sec
2022-12-24T19:43:51.113
